# 도입

#### 병원 개/폐업 분류 예측 경진대회

#### 2020.12.15  
 
1. baseline code.  
2. 변수 탐색 x -> 결측값만.  
3. ~~~시간 변수 처리 수치화 어떻게 모델에 잘 녹일지. (일단 year,month,day 빼기)~~
4. ~~평균,중앙값 중 어떤걸 대체. (일단 평균)  ~~
5. 모델 선택. (scikit-learn cheet sheet)     
    5-1. LinearSVC 는 안먹힌다. convergen?   
    5-2. SVC 실행 시 [Libsvm] 계속 먹통. -> kernel='linear' 로 해결. 
        5-2-1. kernel : 방법?
    
#### 2020.12.16

~~1. 모델 설명하는 shap로 하고 변수들의 중요도 파악 후 데이터 분석    
    1-1. 위가 맞는지 분석을 하고 모델에 넣고 중요도를 파악하는게 맞는지.  
    1-2. shap 로 했지만, 교차검증으로 한 모델들은 클래스를 한개밖에 못읽는다. 그렇게 된거? ~~
    
~~2. shap로 나온 특성들을 우선 분석.~~

#### 2020.12.18

1. 코드 공유를 봤는데 간단한 전처리만 했다.
2. 의사 결정 트리가 생각보다 쎈거같다.
3. 각각의 모델마다 다른 결과값들이 나온다. 그래서 합해야 하는것 같다. 다음엔 randomforest로 하기.

#### 2020.12.21

1. 모델   
    1-1. 의사결정트리    
    1-2. naive bayes    
    1-3. support vector machine.  
    1-4. logistic regression.  
    1-5. decision tree.
        * 문제점 : 왜인지 같은 점수가 나온다. 비슷해서?
2. 스케일
    2-1. ~~minmax 사용. -> 레이블값도 0,1 로 다같이 스케일을 통일.~~
        * scale 을 사용하는 이유 : 모델에도 계수가 있다고 한다. 특성마다 가질 수 있는 값의 스케일이 크게 차이가 나면 문제가 생기기도 한다. 정형화(regulation)는 과학습(overfitting)이 발생하는 것을 방지하기 위한 장치다. 과학습이 발생하면 특정 입력값에 대한 머신러닝 모델의 계수가 커진다. 하지만 입력값의 열마다 가질 수 있는 값의 범위에 크게 차이가 생기면 과학습의 발생 여부와 상관없이 각 입력값에 모델 계수 스케일도 크게 차이가 생겨서 정형화가 정상적으로 작동하지 않는다.
     
#### ~~2020.12.22~~ 2020.12.23(수정)

1. hold out - 전체를 train 과 test 로 나눠서 훈련. 여기서 발전이 k-fold 
2. one-hot encoding vs label encoding  
    * label 인코딩은 rank 를 생각하고 한게 아닌데도 숫자개념때문에 순서가 생긴다. 
        * 순서가 있을때 하면 좋다. 
        * 내용이 많이 있을때 하면 좋다. 
    * one hot 인코딩은 더미 트랩을 일으켜 다중공선성(독립 특성간 종속성)이 발생한다.
        * 순서가 없을때 
        * 내용이 많이 없을때 하면 좋다.
3. smote 
    * 오버샘플링,언더샘플링 기법. 
    * lightgbm 실험가치 있다.

4. 다중대입법.

#### 2020.12.23(수정)

1. xgboost의 장점
    - ~~결측값 자동처리 - 나머지 모델들도 가능한지?~~ 아직 실험중 1.13.0 버전
    - ~~xgboost random_state 가 따로없다. colsample_bytree 와 sumsample 로 해결.~~ seed 가있었다.

#### 2020.12.24

1. 우승자코드 분석.
    1-1. 1등 
        train,test 특히 범주형 column의 level 맞춰주기 -> train,test 환경이 비슷해야한다. 같이 인코딩하기
        R은 factor형도 xgboost에 가능한거 같다. -> 범주형 변수 처리 고민.
        여기는 데이터가 적은데 깊이 10에 num_boost_round 2000으로 땡겼다. -> 물어보기.
        regression 사용.
        파생변수 -> 2016년과 2017년 차이.

In [1]:
# 필요한 라이브러리

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# 데이터 불러오기 

train=pd.read_csv('../data02/train.csv')
test=pd.read_csv('../data02/test.csv')
submission=pd.read_csv('../data02/submission_sample.csv')

In [3]:
# 합치기

n=train.shape[0]
data=pd.concat([train,test],axis=0).reset_index(drop=True)

In [4]:
def datainfo(df):
    return pd.DataFrame([(col,df[col].nunique(),df[col].isna().sum(),df[col].dtype,df[col].unique()[:5]) 
                        for col in df.columns],columns=['name','nunique','missing','dtype','value :5'])

In [5]:
# datainfo(data)

In [6]:
data['employee1']=data['employee1'].astype('str').str.replace(',','').astype('float')
data['employee2']=data['employee2'].astype('str').str.replace(',','').astype('float')

train.csv - 의료기관이 폐업했는지 여부를 포함하여 최근 2개년의 재무정보와 병원 기본정보 

test.csv - 폐업 여부를 제외하고 train.csv와 동일 

sample_submission.csv - inst_id와 open과 close를 예측하는 OC 두개의 열로 구성. OC의 값은 open 예측일 경우 1, close 예측일 경우 0.



inst_id - 각 파일에서의 병원 고유 번호

OC – 영업/폐업 분류, 2018년 폐업은 2017년 폐업으로 간주함

sido – 병원의 광역 지역 정보

sgg – 병원의 시군구 자료

openDate – 병원 설립일

bedCount - 병원이 갖추고 있는 병상의 수

instkind – 병원, 의원, 요양병원, 한의원, 종합병원 등 병원의 종류

·        종합병원 : 입원환자 100명 이상 수용 가능

·        병원 : 입원 환자 30명 이상 100명 미만 수용 가능

·        의원 : 입원 환자 30명 이하 수용 가능

·        한방 병원(한의원) : 침술과 한약으로 치료하는 의료 기관.  

 

revenue1 – 매출액, 2017(회계년도)년 데이터를 의미함

salescost1 – 매출원가, 2017(회계년도)년 데이터를 의미함

sga1 - 판매비와 관리비, 2017(회계년도)년 데이터를 의미함

salary1 – 급여, 2017(회계년도)년 데이터를 의미함

noi1 – 영업외수익, 2017(회계년도)년 데이터를 의미함

noe1 – 영업외비용, 2017(회계년도)년 데이터를 의미함

Interest1 – 이자비용, 2017(회계년도)년 데이터를 의미함

ctax1 – 법인세비용, 2017(회계년도)년 데이터를 의미함

Profit1 – 당기순이익, 2017(회계년도)년 데이터를 의미함

liquidAsset1 – 유동자산, 2017(회계년도)년 데이터를 의미함

quickAsset1 – 당좌자산, 2017(회계년도)년 데이터를 의미함

receivableS1 - 미수금(단기), 2017(회계년도)년 데이터를 의미함

inventoryAsset1 – 재고자산, 2017(회계년도)년 데이터를 의미함

nonCAsset1 – 비유동자산, 2017(회계년도)년 데이터를 의미함

tanAsset1 – 유형자산, 2017(회계년도)년 데이터를 의미함

OnonCAsset1 - 기타 비유동자산, 2017(회계년도)년 데이터를 의미함

receivableL1 – 장기미수금, 2017(회계년도)년 데이터를 의미함

debt1 – 부채총계, 2017(회계년도)년 데이터를 의미함

liquidLiabilities1 – 유동부채, 2017(회계년도)년 데이터를 의미함

shortLoan1 – 단기차입금, 2017(회계년도)년 데이터를 의미함

NCLiabilities1 – 비유동부채, 2017(회계년도)년 데이터를 의미함

longLoan1 – 장기차입금, 2017(회계년도)년 데이터를 의미함

netAsset1 – 순자산총계, 2017(회계년도)년 데이터를 의미함

surplus1 – 이익잉여금, 2017(회계년도)년 데이터를 의미함

 

revenue2 – 매출액, 2016(회계년도)년 데이터를 의미함

salescost2 – 매출원가, 2016(회계년도)년 데이터를 의미함

sga2 - 판매비와 관리비, 2016(회계년도)년 데이터를 의미함

salary2 – 급여, 2016(회계년도)년 데이터를 의미함

noi2 – 영업외수익, 2016(회계년도)년 데이터를 의미함

noe2 – 영업외비용, 2016(회계년도)년 데이터를 의미함

interest2 – 이자비용, 2016(회계년도)년 데이터를 의미함

ctax2 – 법인세비용, 2016(회계년도)년 데이터를 의미함

profit2 – 당기순이익, 2016(회계년도)년 데이터를 의미함

liquidAsset2 – 유동자산, 2016(회계년도)년 데이터를 의미함

quickAsset2 – 당좌자산, 2016(회계년도)년 데이터를 의미함

receivableS2 - 미수금(단기), 2016(회계년도)년 데이터를 의미함

inventoryAsset2 – 재고자산, 2016(회계년도)년 데이터를 의미함

nonCAsset2 – 비유동자산, 2016(회계년도)년 데이터를 의미함

tanAsset2 – 유형자산, 2016(회계년도)년 데이터를 의미함

OnonCAsset2 - 기타 비유동자산, 2016(회계년도)년 데이터를 의미함

receivableL2 – 장기미수금, 2016(회계년도)년 데이터를 의미함

Debt2 – 부채총계, 2016(회계년도)년 데이터를 의미함

liquidLiabilities2 – 유동부채, 2016(회계년도)년 데이터를 의미함

shortLoan2 – 단기차입금, 2016(회계년도)년 데이터를 의미함

NCLiabilities2 – 비유동부채, 2016(회계년도)년 데이터를 의미함

longLoan2 – 장기차입금, 2016(회계년도)년 데이터를 의미함

netAsset2 – 순자산총계, 2016(회계년도)년 데이터를 의미함

surplus2 – 이익잉여금, 2016(회계년도)년 데이터를 의미함

employee1 – 고용한 총 직원의 수, 2017(회계년도)년 데이터를 의미함

employee2 – 고용한 총 직원의 수, 2016(회계년도)년 데이터를 의미함

ownerChange – 대표자의 변동 

### 재무 도메인 지식

<h4>용어정리</h4>

매출액 (=영업수익) : 쉽게 말해 수입 +  
매출원가  : 매출액을 올리는데 들어가는 비용 -  
매출총이익 : 매출액 - 매출원가   
판매비와 관리비 (=판관비) : 유지비용 -   
영업이익 : 매출총이익 - 판관비  
세전이익 : 영업이익 + 영업외수익 - 영업외비용  
영업외손익: 영업 외적으로 생긴 손익 (매출액 대비 영업외손익 수준 파악)  
법인세 : 기업에 부과하는 세   
당기순이익 : 기업의 최종수익 -> 세전이익 - 법인세    
매출액순이익율 : 당기순이익/매출액  
유동자산 : 1년내에 현금화 가능한 자산 (=당좌좌산 + 재고자산)  
당좌자산 : 바로 현금화 가능한 자산 (미수금 등 포함)  
비유동자산 : 현금화에 1년이상 걸리는 자산   
유형자산 : 토지, 건물, 기계장치 같은.   
무형자산 : 특허 등.  
단기차입급 : 1년이내에 갚아야할 자산    
비유동부채 : 만기가 1년이상 이후에 도래하는것. (장기차입급)   
유동성장기부채  
채권발행(사채)    
장기차입급 : 차입급 지급기한이 1년을 초과하는것. 장기적으로 약속할때 쓰임.  
이익잉여금 : 법정자본액을 초과한 금액. 
이자보상배율 : 한해수입 중 이자가 얼마인지 파악하는 지표. (=영업이익/이자비용)  
유동비율 : 1년내에 현금으로 바꿀수있는 자산이 1년내에 갚아야할 부채보다 많은 정도.  

In [7]:
# train['OC'].value_counts()

In [8]:
factor_columns=data.select_dtypes(include=['object']).columns
numeric_columns=data.columns.difference(factor_columns)

data[factor_columns]=data[factor_columns].fillna('notsure')
data[numeric_columns]=data[numeric_columns].fillna(-999)

In [9]:
from sklearn.preprocessing import LabelEncoder

encoder=LabelEncoder()

for col in ['sido','instkind','ownerChange']:
    data[col]=encoder.fit_transform(data[col])

In [10]:
data

,inst_id,OC,sido,sgg,openDate,bedCount,instkind,revenue1,salescost1,sga1,...,debt2,liquidLiabilities2,shortLoan2,NCLiabilities2,longLoan2,netAsset2,surplus2,employee1,employee2,ownerChange
0,1,open,2,73,20071228.0,175.0,5,4.217530e+09,0.000000e+00,3.961135e+09,...,7.589937e+08,2.228769e+08,0.000000e+00,5.361169e+08,3.900000e+08,2.619290e+09,1.271224e+09,62.0,64.0,2
1,3,open,9,32,19970401.0,410.0,2,-9.990000e+02,-9.990000e+02,-9.990000e+02,...,-9.990000e+02,-9.990000e+02,-9.990000e+02,-9.990000e+02,-9.990000e+02,-9.990000e+02,-9.990000e+02,801.0,813.0,2
2,4,open,8,89,20161228.0,468.0,5,1.004522e+09,5.154837e+08,4.472197e+08,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,234.0,1.0,2
3,7,open,10,141,20000814.0,353.0,2,7.250734e+10,0.000000e+00,7.067740e+10,...,3.775501e+10,1.701860e+10,9.219427e+09,2.073641e+10,1.510000e+10,1.295427e+10,7.740829e+09,663.0,663.0,2
4,9,open,9,32,20050901.0,196.0,2,4.904354e+10,0.000000e+00,4.765605e+10,...,5.143259e+10,3.007259e+10,1.759375e+10,2.136001e+10,1.410803e+10,5.561941e+06,9.025550e+09,206.0,197.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423,424,notsure,13,33,20011012.0,0.0,6,2.233031e+10,8.484657e+08,1.849255e+10,...,1.563217e+10,1.288180e+10,0.000000e+00,2.750371e+09,0.000000e+00,1.384958e+10,0.000000e+00,560.0,-999.0,1
424,425,notsure,8,74,20010530.0,-999.0,7,1.833906e+10,0.000000e+00,1.760117e+10,...,3.176628e+09,1.376742e+09,0.000000e+00,1.799886e+09,1.740000e+09,8.574502e+09,7.136681e+09,132.0,137.0,2
425,429,notsure,8,116,20010101.0,150.0,3,1.424266e+10,1.726174e+09,1.148722e+10,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-999.0,-999.0,1
426,430,notsure,11,76,20010201.0,-999.0,4,4.892710e+10,4.157148e+10,4.721485e+09,...,3.720397e+10,8.485918e+09,0.000000e+00,2.871805e+10,2.563120e+10,-2.050629e+08,0.000000e+00,363.0,343.0,2


In [11]:
submission['OC']=1
sub=pd.DataFrame({
    'inst_id':submission['inst_id'],
    'OC':1
})

In [12]:
new_train=data.iloc[:n,:]
new_test=data.iloc[n:,:]

X_train=new_train.drop('OC',axis=1)
y_train=np.where(new_train['OC']=='open',1,0)
X_test=new_test.drop('OC',axis=1)

In [13]:
import xgboost as xgb

dtrain=xgb.DMatrix(X_train,label=y_train)
dtest=xgb.DMatrix(X_test)
params={
    'eta':0.05,
    'objective':'binary:logistic',
    'colsample_bytree':1,
    'subsample':1,
    'eval_metric':'logloss',
    'max_delta_step':1
}

bst=xgb.train(params,dtrain,num_boost_round=500)
sub['OC']=bst.predict(dtest)

In [14]:
idx=sub[sub['OC']<0.5].index
submission.loc[idx,'OC']=1

In [15]:
# fig,ax=plt.subplots(1,1,figsize=(6,7),dpi=200)
# xgb.plot_importance(bst,ax=ax,max_num_features=10)

In [16]:
# bedCount,opendate,ownerChange,noe,interest,profit,employee,newasset,revenue 가 중요해보인다.

In [17]:
# 1. 병원의 크기

col1=['sido','sgg','bedCount','instkind','employee1','employee2',
      'inventoryAsset1','inventoryAsset2','tanAsset1','tanAsset2',
      'nonCAsset1','nonCAsset2','sga1','sga2','salary1','salary2',
      'ctax1','ctax2','OnonCAsset2','OnonCAsset1','OC']
data1=data[col1]

new_train=data1.iloc[:n,1:]
new_test=data1.iloc[n:,1:]

X_train=new_train.drop('OC',axis=1)
y_train=np.where(new_train['OC']=='open',1,0)
X_test=new_test.drop('OC',axis=1)

dtrain=xgb.DMatrix(X_train,label=y_train)
dtest=xgb.DMatrix(X_test)
params={
    'eta':0.05,
    'objective':'binary:logistic',
    'colsample_bytree':1,
    'subsample':1,
    'eval_metric':'logloss',
    'max_delta_step':1
}

bst=xgb.train(params,dtrain,num_boost_round=500)
sub['OC']=bst.predict(dtest)

In [18]:
idx=sub[sub['OC']<0.5].index
submission.loc[idx,'OC']=0

In [19]:
# 2. 병원의 고정자산

col1=['bedCount','instkind','sga1','sga2','salary1','salary2','noe1','noe2',
      'inventoryAsset1','inventoryAsset2','tanAsset1','tanAsset2','ctax1','ctax2',
      'employee1','employee2','OC']
data1=data[col1]

new_train=data1.iloc[:n,1:]
new_test=data1.iloc[n:,1:]

X_train=new_train.drop('OC',axis=1)
y_train=np.where(new_train['OC']=='open',1,0)
X_test=new_test.drop('OC',axis=1)

dtrain=xgb.DMatrix(X_train,label=y_train)
dtest=xgb.DMatrix(X_test)
params={
    'eta':0.05,
    'objective':'binary:logistic',
    'colsample_bytree':1,
    'subsample':1,
    'eval_metric':'logloss',
    'max_delta_step':1
}

bst=xgb.train(params,dtrain,num_boost_round=500)
sub['OC']=bst.predict(dtest)

In [20]:
idx=sub[sub['OC']<0.5].index
submission.loc[idx,'OC']=0

In [21]:
# 병원의 이익

col1=['revenue1','revenue2','salescost1','salescost2','sga1','sga2',
      'noi1','noi2','noe1','noe2','netAsset1','netAsset2','surplus1',
      'surplus2','profit1','profit2','OC']

data1=data[col1]

new_train=data1.iloc[:n,1:]
new_test=data1.iloc[n:,1:]

X_train=new_train.drop('OC',axis=1)
y_train=np.where(new_train['OC']=='open',1,0)
X_test=new_test.drop('OC',axis=1)

dtrain=xgb.DMatrix(X_train,label=y_train)
dtest=xgb.DMatrix(X_test)
params={
    'eta':0.05,
    'objective':'binary:logistic',
    'colsample_bytree':1,
    'subsample':1,
    'eval_metric':'logloss',
    'max_delta_step':1
}

bst=xgb.train(params,dtrain,num_boost_round=500)
sub['OC']=bst.predict(dtest)

In [22]:
idx=sub[sub['OC']<0.5].index
submission.loc[idx,'OC']=0

In [23]:
# 병원의 부채

col1=['receivableS1','receivableS2','receivableL1','receivableL2','debt1','debt2',
      'NCLiabilities1','NCLiabilities2','shortLoan1','shortLoan2','longLoan1','longLoan2',
      'liquidLiabilities1','liquidLiabilities2','OC']

data1=data[col1]

new_train=data1.iloc[:n,1:]
new_test=data1.iloc[n:,1:]

X_train=new_train.drop('OC',axis=1)
y_train=np.where(new_train['OC']=='open',1,0)
X_test=new_test.drop('OC',axis=1)

dtrain=xgb.DMatrix(X_train,label=y_train)
dtest=xgb.DMatrix(X_test)
params={
    'eta':0.05,
    'objective':'binary:logistic',
    'colsample_bytree':1,
    'subsample':1,
    'eval_metric':'logloss',
    'max_delta_step':1
}

bst=xgb.train(params,dtrain,num_boost_round=500)
sub['OC']=bst.predict(dtest)

In [24]:
idx=sub[sub['OC']<0.6].index
submission.loc[idx,'OC']=0

In [25]:
# 각종 지표

import warnings
warnings.filterwarnings("ignore")

col1=['revenue1','revenue2','profit1','profit2','liquidAsset1','liquidAsset2',
      'shortLoan1','shortLoan2','surplus1','surplus2','quickAsset1','quickAsset2',
      'debt1','debt2','netAsset1','netAsset2','OC']

data1=data[col1]

# 유동자산/단기차입금  : 1년이내에 갚을돈이 많은지 정도.
data1['rateofliquid1']=data1['liquidAsset1']/data1['shortLoan1']
data1.loc[data1['rateofliquid1']==np.inf,'rateofliquid1']=0
data1.loc[data1['rateofliquid1'].isna(),'rateofliquid1']=-999

# 총자산증가율
# data1['rateofasset']=data1['netAsset1']/data1['netAsset2']-1
# data1.loc[data1['rateofasset']==-1,'rateofasset']=-999

# 총매출증가율
# data1['rateofrevenue']=data1['revenue1']/data1['revenue2']-1
# data1.loc[data1['rateofrevenue']==-1,'rateofrevenue']=-999

# 자산/부채+자산 비율
data1['rateofownasset1']=1/(1+(data1['debt1']/data1['netAsset1']))
data1.loc[data1['rateofownasset1'].isna(),'rateofownasset1']=-999

# 이익 %
data1['rateofprofit1']=data1['profit1']/(data1['revenue1']+1)

# 정리
data1=data1.drop(['liquidAsset2','revenue2','profit2','shortLoan2','surplus2',
            'quickAsset2','debt2','netAsset2'],axis=1)

In [26]:
new_train=data1.iloc[:n,1:]
new_test=data1.iloc[n:,1:]

X_train=new_train.drop('OC',axis=1)
y_train=np.where(new_train['OC']=='open',1,0)
X_test=new_test.drop('OC',axis=1)

dtrain=xgb.DMatrix(X_train,label=y_train)
dtest=xgb.DMatrix(X_test)
params={
    'eta':0.05,
    'objective':'binary:logistic',
    'colsample_bytree':1,
    'subsample':1,
    'eval_metric':'logloss',
    'max_delta_step':1
}

bst=xgb.train(params,dtrain,num_boost_round=500)
sub['OC']=bst.predict(dtest)

In [27]:
idx=sub[sub['OC']<0.6].index
submission.loc[idx,'OC']=0

In [28]:
# submission.to_csv('../submission/xgboost(step)_2020.12.23.csv',index=False)